## 生成式对话机器人

### Step1 导入相关包

In [2]:
import os

# 设置可见的 GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "4,5,7"

from datasets import  Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

### Step2 加载数据集

In [3]:
ds = Dataset.load_from_disk("alpaca_data_zh")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [6]:
ds[:3]

{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
  '4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。',
  '朱利叶斯·凯撒，又称尤利乌斯·恺撒（Julius Caesar）是古罗马的政治家、军事家和作家。他于公元前44年3月15日被刺杀。 \n\n根据历史记载，当时罗马元老院里一些参议员联合起来策划了对恺撒的刺杀行动，因为他们担心恺撒的统治将给罗马共和制带来威胁。在公元前44年3月15日（又称“3月的艾达之日”），恺撒去参加元老院会议时，被一群参议员包围并被攻击致死。据记载，他身中23刀，其中一刀最终致命。'],
 'input': ['', '输入：4/16', ''],
 'instruction': ['保持健康的三个提示。', '解释为什么以下分数等同于1/4', '朱利叶斯·凯撒是如何死亡的？']}

### Step3 数据集处理

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-389m-zh")
tokenizer

tokenizer_config.json:   0%|          | 0.00/268 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

BloomTokenizerFast(name_or_path='Langboat/bloom-389m-zh', vocab_size=42437, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [8]:
def process_function(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human:" + example["instruction"], example["input"]]).strip() + "\n\nAssistant:")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [9]:
tokenized_ds = ds.map(process_function, remove_columns=ds.column_names)
tokenized_ds

Map:   0%|          | 0/26858 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [12]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

'Human:解释为什么以下分数等同于1/4\n输入：4/16\n\nAssistant:4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

In [13]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

### Step4 创建模型

In [14]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-389m-zh")

config.json:   0%|          | 0.00/431 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

### Step5 配置训练参数

In [15]:
import logging

logging.basicConfig(level=logging.INFO)

In [16]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=3
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Step7 创建训练器

In [17]:
trainer = Trainer(
    args=args,
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

### Step8 模型训练

In [18]:
trainer.train()

/node6_1/tanshuai/.conda/envs/abc/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
10,2.983500
20,2.916200
30,2.854700
40,2.810300
50,2.760900
60,2.681100
70,2.651200
80,2.575300
90,2.526400
100,2.551400


/node6_1/tanshuai/.conda/envs/abc/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=837, training_loss=1.7871749016546434, metrics={'train_runtime': 1403.0683, 'train_samples_per_second': 57.427, 'train_steps_per_second': 0.597, 'total_flos': 2.0973211157938176e+16, 'train_loss': 1.7871749016546434, 'epoch': 2.9906208128628853})

### Step9 模型推理

In [19]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

In [41]:
input = "Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant:"
print(pipe(input, max_length=256, do_sample=True, top_k=5, temperature=0.8))

[{'generated_text': 'Human: 考试有哪些技巧？\n\nAssistant:考试有很多技巧，以下是其中一些常见技巧的示例：\n\n1. 复习策略：复习可以分为三步走：第一步是熟悉考试内容和题型，这可以通过阅读和练习来完成；第二步是准备答题技巧，包括选择正确的题型、运用正确的解题步骤、分析答案等；第三步是准备作弊技巧，如作弊软件、作弊材料等。\n\n2. 时间管理：有效的时间管理可以确保考生在规定时间内完成考试内容和答题技巧。这可以通过设置考前复习计划、合理安排作息时间等方法来实现。\n\n3. 考前准备：考前准备可以确保考生熟悉考场环境、熟悉考试要求，并能够有效地应对考试。这可以通过提前做准备题、认真阅读考试说明、做好考前准备材料等方法来实现。\n\n4. 心理调适：心理调节可以有效缓解压力，有助于考生在考场上获得更好的表现。这可以通过提供适当的考前心理指导和帮助，以及保持良好的作息时间等方法来实现。\n\n以上是一些常见技巧，希望能对您有所帮助。'}]
